In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
# Locally
# data_url = 'https://github.com/dmika1234/ml_uwr_22/blob/Project/Project/data/fake_job_postings.csv'
data_path = 'data/fake_job_postings.csv'
raw_data = pd.read_csv(data_path)

# For colab
# data_url = '/content/fake_job_postings.csv'
# raw_data = pd.read_csv(data_url, error_bad_lines=False, engine="python")
#straszny problem miałem, żeby wczytać te dane tak ja ty to robiłeś. dziwne błędy mi wyskakiwały

In [4]:
text_colnames = ['company_profile', 'description', 'requirements', 'benefits']
text_data = raw_data[['job_id'] + text_colnames]
text_data.fillna('', inplace=True)

tokenizer = RegexpTokenizer(r'\w+')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for i in range(text_data.shape[0]):
    for col in text_colnames:
        text_data.at[i, col] = tokenizer.tokenize(text_data.at[i, col].lower())  # removing punctuation
        filtered_sentence = []
        for w in text_data.at[i, col]:
            if w not in stop_words:  # removing words such as “the”, “a”, “an”, “in”
                filtered_sentence.append(
                    lemmatizer.lemmatize(w))  # grouping together the different inflected forms of a word
        text_data.at[i, col] = list(filtered_sentence)

C:\Users\dmika\AppData\Local\Temp\ipykernel_8976\4157831128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data.fillna('', inplace=True)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Word2vec playground

### Simple example

In [31]:
sentences = [['this', 'is', 'a', 'good', 'example'], ['this', 'is', 'another', 'example']]
model = Word2Vec(sentences, min_count=1)
model.wv['example']

array([-5.3622725e-04,  2.3643136e-04,  5.1033497e-03,  9.0092728e-03,
       -9.3029495e-03, -7.1168090e-03,  6.4588725e-03,  8.9729885e-03,
       -5.0154282e-03, -3.7633716e-03,  7.3805046e-03, -1.5334714e-03,
       -4.5366134e-03,  6.5540518e-03, -4.8601604e-03, -1.8160177e-03,
        2.8765798e-03,  9.9187379e-04, -8.2852151e-03, -9.4488179e-03,
        7.3117660e-03,  5.0702621e-03,  6.7576934e-03,  7.6286553e-04,
        6.3508903e-03, -3.4053659e-03, -9.4640139e-04,  5.7685734e-03,
       -7.5216377e-03, -3.9361035e-03, -7.5115822e-03, -9.3004224e-04,
        9.5381187e-03, -7.3191668e-03, -2.3337686e-03, -1.9377411e-03,
        8.0774371e-03, -5.9308959e-03,  4.5162440e-05, -4.7537340e-03,
       -9.6035507e-03,  5.0072931e-03, -8.7595852e-03, -4.3918253e-03,
       -3.5099984e-05, -2.9618145e-04, -7.6612402e-03,  9.6147433e-03,
        4.9820580e-03,  9.2331432e-03, -8.1579173e-03,  4.4957981e-03,
       -4.1370760e-03,  8.2453608e-04,  8.4986202e-03, -4.4621765e-03,
      

In [32]:
model.wv.index_to_key

['example', 'is', 'this', 'another', 'good', 'a']

### Our data 1st try

In [12]:
# Train Word2Vec model on job description column
model2 = Word2Vec(text_data['description'], vector_size=20, min_count=1)

ValueError: need at least one array to concatenate

In [13]:
model2.wv['growing']

array([-6.825443  ,  3.7083013 ,  4.4011292 ,  9.217835  ,  0.8840265 ,
        3.5300004 , -0.9777564 ,  3.3203785 , -0.15862823, -0.8004102 ,
       -1.1629566 ,  3.3845246 ,  2.9411807 , -3.500717  , -0.44814137,
       -1.4070075 ,  5.3038187 , -2.2519035 ,  0.0648962 ,  1.355451  ],
      dtype=float32)

In [14]:
model2.wv.most_similar(positive='growing')

[('expanding', 0.8730002641677856),
 ('upsurge', 0.753235936164856),
 ('teamrepresent', 0.7406718730926514),
 ('unbeatable', 0.734790563583374),
 ('transpoco', 0.7288982272148132),
 ('spacedelivering', 0.7272077798843384),
 ('engineerwork', 0.7136224508285522),
 ('brandsensure', 0.7047767043113708),
 ('500equinix', 0.6980654001235962),
 ('paced', 0.6943872570991516)]

### Pretrained model on Google news data

https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [ ]:
!pip install gdown
gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [17]:
from gensim.models import KeyedVectors

model_google = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [20]:
model_google.most_similar(positive='growing')

[('grown', 0.6805573105812073),
 ('burgeoning', 0.677505373954773),
 ('grow', 0.6641595959663391),
 ('rapidly_expanding', 0.66228848695755),
 ('increasing', 0.6527025699615479),
 ('grows', 0.6311836838722229),
 ('Growing', 0.6133630871772766),
 ('expanding', 0.5987905263900757),
 ('rising', 0.5733980536460876),
 ('growning', 0.5670080780982971)]

array([ 1.44531250e-01, -1.23901367e-02,  1.19140625e-01,  4.15039062e-02,
       -2.00195312e-01, -1.73828125e-01, -4.73632812e-02,  1.78222656e-02,
       -1.22070312e-01,  1.36718750e-02, -6.10947609e-06,  6.73828125e-02,
        2.51464844e-02, -7.03125000e-02, -5.76171875e-02,  4.12597656e-02,
        1.54296875e-01,  6.78710938e-02, -5.09643555e-03, -1.61132812e-01,
        2.58789062e-02, -5.79833984e-03,  1.41601562e-01,  4.88281250e-02,
        1.66015625e-01, -9.96093750e-02, -2.17285156e-02,  8.93554688e-02,
        1.81640625e-01,  4.91333008e-03, -1.53198242e-02,  9.66796875e-02,
        1.85546875e-01, -9.37500000e-02, -1.28173828e-02,  3.27148438e-02,
        1.08032227e-02, -1.64062500e-01,  4.00390625e-02,  9.13085938e-02,
        2.42187500e-01, -8.59375000e-02,  2.73437500e-01,  6.64062500e-02,
       -1.64794922e-02,  7.86132812e-02, -2.12890625e-01,  2.88085938e-02,
        1.17675781e-01,  2.35351562e-01,  3.12500000e-01,  5.88378906e-02,
       -1.57226562e-01, -